In [ ]:
from cStringIO import StringIO
import gzip
import os
import glob
import re
import pdb

import gcp
from gcp import storage
from gcp import bigquery as bq
import pandas as pd

# Import zip file from Google Cloud Storage
project = gcp.Context.default().project_id
# project = 'datalab-dot-alien-oarlock-130015'
bucket_name = project + '-datalab'
bucket_path = 'gs://' + bucket_name + '/data'
print 'Bucket: ' + bucket_path

In [ ]:
# Downloading and extracting the files from cloud storage.
​
def process_datafile(localpath, storagepath, table, mode='create', overwrite=False):
  gzip_filename = 'citydata.tar.gz'
  tar_filename = 'citydata.tar'
  datadir = 'season_1'
  storagepath_r = storagepath.split('/')
  # If data has not been extracted, extract it.
  if not os.path.isdir(datadir):
    # If citydata.tar has not been downloaded, download it
    if not os.path.isfile(tar_filename):
      
      bucket_object = '/'.join(storagepath_r[:3]) + '/citydata.tar.gz'
      %storage read --object $bucket_object --variable compressed_file
      
      gzip_file = gzip.GzipFile(fileobj=StringIO(compressed_file))
      del compressed_file

      import shutil
      with open(tar_filename, 'wb') as f_out:
        shutil.copyfileobj(gzip_file, f_out)

    import tarfile
    tar = tarfile.open(tar_filename, "r")
    tar.extractall()
    tar.close()
    os.remove(tar_filename)

  # Upload extracted file into GCS
  bucketname = storagepath_r[2]
  itempath = '/'.join(storagepath_r[3:])
  item = storage.Item(bucketname, itempath)
  if not item.exists() or overwrite:    
    with open(localpath, 'rb') as f:
      item.write_to(f.read(), 'text/plain')
  # Load data into Google BigQuery
  table.load(storagepath, mode=mode, csv_options=bq.CSVOptions(delimiter='\t'))

# Process Training Data

## Districts

In [ ]:
if not table.exists():
  schema = bq.Schema.from_data(pois_schema)
  table.create(schema)
  
  pois_data = pd.DataFrame(columns=['district_hash'] + pois)
  with open(localpath, 'rb') as f:
    for line in f:
      hash_pois = {}
      for poi_line in line.split('\t')[1:]:
        hash_pois['f{}'.format(poi_line.split(':')[0].replace('#', '_'))] = poi_line.split(':')[1]
      poi_data = [line.split('\t')[0]]
      # hash_pois = {f1_1: 15, ...}
      # pois = ['f1_1', ...]
      for poi in pois:
        value = '0'
        if poi in hash_pois:
          value = hash_pois[poi].strip()
        poi_data.append(value)
      pois_data.loc[len(pois_data)] = poi_data
  for poi in pois:
    pois_data[poi] = pd.to_numeric(pois_data[poi])
  table.insert_data(pois_data)

Following code can be used to print out feature fields to be used when selecting from tables.

In [ ]:
final_text = ''
for counter, poi_text in enumerate(map(lambda x: 'FIRST(pois.{}) AS {}'.format(x,x), pois)):
  if counter%3 == 0:
    final_text = '{}\n'.format(final_text)
  final_text = '{}{}, '.format(final_text, poi_text)
print final_text[1:(len(final_text)-2)]